In [ ]:
from torch.optim import lr_scheduler
import sys
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import torch.optim as optim
from tqdm import tqdm

# 加载规范化数据集
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

train_dataset = ImageFolder(r'/Users/kkkkshi/Desktop/ANLY_677/Project/Project_3/projectdataset/masked/train', transform=train_transform)
val_dataset = ImageFolder(r'/Users/kkkkshi/Desktop/ANLY_677/Project/Project_3/projectdataset/masked/test', transform=val_transform)
train_num =  len(train_dataset)
val_num = len(val_dataset)

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
print("Loading completed")

# vgg model
class VGG(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=False):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        # N x 3 x 224 x 224
        x = self.features(x)
        # N x 512 x 7 x 7
        x = torch.flatten(x, start_dim=1)
        # N x 512*7*7
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                # nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_features(cfg: list):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [conv2d, nn.ReLU(True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


def vgg(model_name="vgg16", **kwargs):
    assert model_name in cfgs, "Warning: model number {} not in cfgs dict!".format(model_name)
    cfg = cfgs[model_name]

    model = VGG(make_features(cfg), **kwargs)
    return model

# train
def train(dataloader, model, loss_fn, optimizer):
    loss, current, n = 0.0, 0.0, 0
    for batch, (x, y) in enumerate(dataloader):
        image, y = x.to(device), y.to(device)
        output = model(image)
        cur_loss = loss_fn(output, y)
        _, pred = torch.max(output, axis=1)
        cur_acc = torch.sum(y==pred) / output.shape[0]

        # 反向传播
        optimizer.zero_grad()
        cur_loss.backward()
        optimizer.step()
        loss += cur_loss.item()
        current += cur_acc.item()
        n = n+1

    train_loss = loss / n
    train_acc = current / n
    print('train_loss' + str(train_loss))
    print('train_acc' + str(train_acc))
    return train_loss, train_acc

# 定义一个验证函数
def val(dataloader, model, loss_fn):
    # 将模型转化为验证模型
    model.eval()
    loss, current, n = 0.0, 0.0, 0
    with torch.no_grad():
        for batch, (x, y) in enumerate(dataloader):
            image, y = x.to(device), y.to(device)
            output = model(image)
            cur_loss = loss_fn(output, y)
            _, pred = torch.max(output, axis=1)
            cur_acc = torch.sum(y == pred) / output.shape[0]
            loss += cur_loss.item()
            current += cur_acc.item()
            n = n + 1

    val_loss = loss / n
    val_acc = current / n
    print('val_loss' + str(val_loss))
    print('val_acc' + str(val_acc))
    return val_loss, val_acc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))

model_name = "vgg11"
vggnet = vgg(model_name=model_name, num_classes=1000, init_weights=True)
vggnet.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vggnet.parameters(), lr=0.0001, momentum=0.9)
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
checkpoint = torch.load(r"/Users/kkkkshi/Desktop/ANLY_677/Project/Project_3/vgg11-bbd30ac9.pth")
vggnet.load_state_dict(checkpoint)
loss_train = []
acc_train = []
loss_val = []
acc_val = []
epochs = 20
for t in range(epochs):
    lr_scheduler.step()
    print(f"epoch{t+1}\n-----------")
    train_loss, train_acc = train(train_dataloader, vggnet, loss_function, optimizer)
    val_loss, val_acc = val(val_dataloader, vggnet, loss_function)

    loss_train.append(train_loss)
    acc_train.append(train_acc)
    loss_val.append(val_loss)
    acc_val.append(val_acc)

    # 保存最好的模型权重
    # if val_acc >min_acc:
    print(f"save model, 第{t+1}轮")
    torch.save(vggnet.state_dict(), r'/Users/kkkkshi/Desktop/ANLY_677/Project/Project_3/proj/vgg/save_model_mask/epoch_'+str(t)+'_best_model.pth')


print('Finished Training')


Loading completed
using cpu device.


/opt/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


epoch1
-----------


In [6]:
#test
print('start test')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
data_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load image
dataset = ImageFolder(r'/Users/kkkkshi/Desktop/ANLY_677/Project/Project_3/projectdataset/masked/test', transform=train_transform)
datasetloader = DataLoader(dataset, batch_size=32, shuffle=True)

# create model
model = vgg(model_name="vgg11", num_classes=1000).to(device)
weights_path = r"/Users/kkkkshi/Desktop/ANLY_677/Project/Project_3/proj/vgg/save_model_mask/epoch_19_best_model.pth"
model.load_state_dict(torch.load(weights_path, map_location=device))
model.eval()

top1 =0
top5= 0
data_num = len(dataset)
acc_num =0;
wrong_num = 0;
for i in range(data_num):
    x, y = dataset[i][0], dataset[i][1]
    x = torch.unsqueeze(x, dim=0)

    with torch.no_grad():
        # predict class
        output = torch.squeeze(model(x.to(device))).cpu()
        predict = torch.softmax(output, dim=0)

        predict_cla = torch.argmax(predict).numpy()
        print("predict:",predict_cla,"actual:",y)
        if predict_cla == y:
            acc_num +=1
wrong_num = data_num-acc_num
print("正确数量：",acc_num)
print("错误数量：",wrong_num)


def evaluteTop1(model, loader):
    model.eval()

    correct = 0
    total = len(loader.dataset)

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
            correct += torch.eq(pred, y).sum().float().item()
    return correct / total


def evaluteTop5(model, loader):
    model.eval()
    correct = 0
    total = len(loader.dataset)
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            maxk = max((1, 5))
            y_resize = y.view(-1, 1)
            _, pred = logits.topk(maxk, 1, True, True)
            correct += torch.eq(pred, y_resize).sum().float().item()
    return correct / total
print("top1 accuracy: ",evaluteTop1(model,datasetloader))
print("top5 accuracy: ",evaluteTop5(model,datasetloader))

start test
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
pr

predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 ac